In [ ]:
! pip install --upgrade --quiet accelerate bitsandbytes transformers

In [ ]:
import io
import os
import gc
import time
import json
import torch
from tqdm import tqdm
from PIL import Image
from google.colab import drive
from huggingface_hub import login
from IPython.display import Image as IPImage, display, Markdown # parte mais visual
from transformers import AutoTokenizer, AutoModelForImageTextToText, AutoProcessor, BitsAndBytesConfig, AutoModelForCausalLM

---
## MedGemma Class

Classe para configuração e uso dos modelos da Google, MedGemma.

### Version1

In [ ]:
class UsingMedGemma():

  models = {
      'pre-trained' : 'google/medgemma-4b-pt',
      'instruct-tuned' : 'google/medgemma-4b-it',
      '27b-multimodal' : 'google/medgemma-27b-it', # versão Multimodal
      '27b-text-only' : 'google/medgemma-27b-text-it' # versão Texto
  }

  def __init__(self, model_version='instruct-tuned', device:str=None, quantization:bool=False):

    # verificação do device
    self.device = self.select_device(device)
    # obtém versão do MedGemma
    self.model_version = self.models[model_version]
    # selecionando parâmetros do modelo
    self.model_params = self.config_model_params(self.device, quantization)

    # seleção da classe para o modelo
    # versão 4B
    if '4b' in self.model_version.lower():
      # inicializando o modelo multimodal
      self.model = AutoModelForImageTextToText.from_pretrained(
          self.model_version,
          **self.model_params
      )
      # inicializando o processor
      self.processor = AutoProcessor.from_pretrained(
        self.model_version,
        use_fast=False
      )
    # versão 27B-text-only
    else:
      self.model = AutoModelForCausalLM.from_pretrained(
          self.model_version,
          **self.model_params # devemos modificar os parâmetros
      )
      self.processor = AutoTokenizer.from_pretrained( # Tokenizer (only-text)
          self.model_version
      )


  def send_only_text(self, text:str, sys_prompt:str=None, max_tokens:int=300, show_result:bool=True) -> str:
    ''' Geração de texto com MedGemma tendo como entrada apenas texto'''
    # estrutura de mensagem
    message = [
        {
            'role': 'user',
            'content': [
                {
                    'type': 'text',
                    'text': text
                }
            ]
        }
    ]
    # inserir prompt de sistema (opcional)
    if sys_prompt is not None:
      message.insert(0, {
          'role': 'system',
          'content': [
              {
                  'type': 'text',
                  'text': sys_prompt
              }
          ]
      })

    # Preparação da entrada para o modelo
    # versão para 4B
    if "4b" in self.model_version.lower():
      inputs = self.processor.apply_chat_template(
          message,
          add_generation_prompt=True,
          tokenize=True,
          return_dict=True,
          return_tensors="pt",
      ).to(self.model.device)
      # anotação : bfloat16 apenas para modelos multimodais
    # versão para 27B
    else:
      inputs = self.processor.apply_chat_template(
          message,
          add_generation_prompt=True,
          tokenize=True,
          return_dict=True,
          return_tensors="pt",
      ).to(self.model.device)

    # tamanho da entrada (para removermos dos tokens de saída)
    input_len = inputs["input_ids"].shape[-1]

    # geração
    with torch.inference_mode():
      generation = self.model.generate(**inputs, max_new_tokens=max_tokens, do_sample=False)
      generation = generation[0][input_len:]

    # decodificação
    answer = self.processor.decode(generation, skip_special_tokens=True)
    # visualização
    if show_result:
      self.show_results(text, answer)

    return answer

  def send_image_and_text(self, text, image_inputs, sys_prompt=None, max_tokens=256, show_result=True):
    """Geração de texto com MedGemma tendo como entrada texto e imagens."""
    if "27b" in self.model_version.lower():
      raise ValueError("[ERRO] Essa função é inválida para esse modelo. O modelo MedGemma-27B é somente texto e não aceita imagens.")

    # Garante que image_inputs é uma lista
    if not isinstance(image_inputs, list):
        image_inputs = [image_inputs]

    # Carrega todas as imagens
    images = []
    for img_input in image_inputs:
        img = self.load_image(img_input)
        if img is not None:
            images.append(img)
        else:
            print(f"[ERRO] Falha ao carregar: {img_input}")

    if not images:
        raise ValueError("[ERRO] Nenhuma imagem válida foi carregada. Abortando requisição.")

    message = [
        {
            'role': 'user',
            'content': [
                {
                    'type': 'text',
                    'text': text
                }
            ]
        }
    ]
    if sys_prompt is not None:
      message.insert(0, {
          'role': 'system',
          'content': [
              {
                  'type': 'text',
                  'text': sys_prompt
              }
          ]
      })

    # Adiciona imagens ao prompt
    pos = 0 if sys_prompt is None else 1
    for img in images:
      message[pos]["content"].append({"type": "image", "image": img})

    inputs = self.processor.apply_chat_template(
        message,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(self.model.device, dtype=torch.bfloat16)

    # Geração
    outputs = self.model.generate(**inputs, max_new_tokens=max_tokens)
    response = self.processor.decode(outputs[0], skip_special_tokens=True)

    if show_result:
        print("\n[RESPOSTA DO MODELO]:\n", response)

    return response

  def load_image(self, image_input):
    """
      Carrega uma ou mais imagens (a partir de caminho, bytes ou BytesIO)
      e converte todas para RGB (formato necessário para o MedGemma).
      Retorna uma única imagem (PIL.Image) ou uma lista de imagens, conforme o input.
    """
    def _load_single_image(single_input):
        """Carrega uma única imagem e converte para RGB."""
        if isinstance(single_input, (str, os.PathLike)):
            if not os.path.exists(single_input):
                raise FileNotFoundError(f"O arquivo de imagem '{single_input}' não foi encontrado.")
            return Image.open(single_input).convert('RGB')

        elif isinstance(single_input, (bytes, bytearray, io.BytesIO)):
            if isinstance(single_input, (bytes, bytearray)):
                single_input = io.BytesIO(single_input)
            return Image.open(single_input).convert('RGB')

        else:
            raise TypeError("Cada elemento deve ser um caminho (str) ou um objeto de arquivo em memória.")

    try:
        # Caso o input seja uma lista ou tupla de imagens
        if isinstance(image_input, (list, tuple)):
            images = []
            for idx, img in enumerate(image_input):
                try:
                    loaded = _load_single_image(img)
                    images.append(loaded)
                except Exception as e:
                    print(f"[ERRO] Falha ao carregar imagem {idx + 1}: {e}")
            return images  # lista de imagens

        # Caso seja apenas uma imagem
        else:
            return _load_single_image(image_input)

    except Exception as e:
        print(f"[ERRO] Falha ao carregar imagem(s): {e}")
        return None

  def show_results(self, text, answer, image=None) -> None:
    display(Markdown(f"---\n\n**[ Requisição ]**\n\n{text}"))
    if image:
      if isinstance(image, list):
        for idx, img in enumerate(image, start=1):
          display(Markdown(f"**Imagem {idx}:**"))
          display(img)
      else:
          display(image)
    display(Markdown(f"---\n\n**[ MedGemma ]**\n\n{answer}\n\n---"))

  def select_device(self, device: str = None) -> str:
    """
      Verifica se o device está disponível e retorna a string correspondente.
      Caso device seja None, tenta usar GPU (CUDA) se disponível, senão CPU.
      Exibe avisos recomendando o uso de GPU.
    """
    # Caso o usuário tenha especificado manualmente
    if device is not None:
      if device.startswith("cuda") and not torch.cuda.is_available():
        print("[AVISO] CUDA foi especificado, mas nenhuma GPU está disponível. Alternando para CPU.")
        return "cpu"
      elif device == "cpu":
        print("[AVISO] O modelo será executado na CPU. Isso pode ser significativamente mais lento.")
        print("[SUGESTÃO] Considere usar uma GPU (CUDA) para acelerar a inferência.")
        return "cpu"
      else:
        return device

    # Seleção automática
    if torch.cuda.is_available():
      print("[INFO] GPU CUDA detectada. Utilizando GPU para melhor desempenho.")
      return "cuda"
    else:
      print("[AVISO] Nenhuma GPU detectada. O modelo será executado na CPU.")
      print("[SUGESTÃO] Considere usar uma GPU (CUDA) para acelerar a execução.")
      return "cpu"

  def config_model_params(self, device:str, use_quantization:bool) -> dict:
    # Definir os parâmetros do modelo baseado no ambiente do Colab

    # Caso dos modelos de 27B
    if '27b' in self.model_version:
      if 'A100' in torch.cuda.get_device_name(0):
        # Parâmetros utilizados pelo exemplo da própria Google
        model_kwargs = dict(
            torch_dtype=torch.bfloat16,
            device_map='auto',
            quantization_config=BitsAndBytesConfig(
                load_in_4bit=True # diminuição dos pesos para 4bits
            )
        )
      else:
        raise ValueError(
            "Runtime has insufficient memory to run a 27B variant."
            "Please select an A100 GPU."
        )
    # Caso dos modelos de 4B
    elif '4b' in self.model_version:
      if device == 'cpu': # apenas para depurar
        # Nesse modo a RAM fica muito próxima do seu limite
        model_kwargs = dict(
            device_map='cpu',
            # torch_dtype='float32',  # precisão máxima (está estourando o limite da RAM, 12.7GB)
            torch_dtype=torch.float16, # Deve utilizar essa precisão para não estourar a RAM
            low_cpu_mem_usage=True  # otimiza carregamento
        )
      elif device == 'cuda' and not use_quantization:
        # uso estimado de 10~12GB de VRAM
        # Ao utilizar esse modo a VRAM atingiu ~11GB de 15GB
        model_kwargs = dict(
            device_map='auto',
            torch_dtype=torch.bfloat16  # mais leve que float32
        )
      elif device == 'cuda' and use_quantization:
        # uso estimado de 4~6GB de VRAM
        # Utiliza ~4GB da VRAM
        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type='nf4'
        )
        model_kwargs = dict(
            device_map='auto',
            quantization_config=quant_config
        )
      else:
        print(f'No valid option')
        return None

    print(f'Parâmetros do Modelo:\n{model_kwargs=}')
    return model_kwargs

###Version2

In [ ]:
class MedGemma():

  models = {
      '4b-pt' : 'google/medgemma-4b-pt',
      '4b-it' : 'google/medgemma-4b-it',
      '27b-mult' : 'google/medgemma-27b-it', # versão Multimodal
      '27b-text' : 'google/medgemma-27b-text-it' # versão Texto
  }

  '''

    * função específica que processa os dados (tanto textos quanto imagens)

    * função para aceitar entrada dict e retornar dict com resposta

  '''

  def __init__(self, model_version:str='4b-it', device:str=None,
             quantization:bool=False, model=None, processor=None):

    self.device = self.select_device(device)
    self.model_version = self.models[model_version]
    self.model_params = self.config_model_params(self.device, quantization)

    # Modelo
    if model is not None:
        self.model = model
        print(f"[INFO] Using preloaded model: {getattr(model, 'name_or_path', 'unknown')}")
    else:
        print(f"[INFO] Initializing model: {self.model_version}")
        if '4b' in self.model_version.lower():
            self.model = AutoModelForImageTextToText.from_pretrained(
                self.model_version,
                **self.model_params
                )
        elif '27b' in self.model_version.lower():
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_version,
                **self.model_params
                )
        else:
            raise ValueError(f"[ERROR] Invalid model version: {self.model_version}")

    # Processor
    if processor is not None:
        self.processor = processor
        print(f"[INFO] Using preloaded processor: {getattr(processor, 'name_or_path', 'unknown')}")
    else:
        print(f"[INFO] Initializing processor: {self.model_version}")
        if '4b' in self.model_version.lower():
            self.processor = AutoProcessor.from_pretrained(
                self.model_version,
                use_fast=False
                )
        elif '27b' in self.model_version.lower():
            self.processor = AutoTokenizer.from_pretrained(
                self.model_version
                )
        else:
            raise ValueError(f"[ERROR] Invalid processor version: {self.model_version}")

  def return_model(self):
    return self.model

  def return_processor(self):
    return self.processor

  def processor_data(self, message, dtype=None):
    inputs = self.processor.apply_chat_template(
          message,
          add_generation_prompt=True,
          tokenize=True,
          return_dict=True,
          return_tensors="pt",
      )

    if dtype:
      return inputs.to(self.model.device, dtype=dtype)

    return inputs.to(self.model.device)

  def send_only_text(self, text:str, sys_prompt:str=None, max_tokens:int=300, show_result:bool=True) -> str:
    ''' Geração de texto com MedGemma tendo como entrada apenas texto'''
    # estrutura de mensagem
    message = [
        {
            'role': 'user',
            'content': [
                {
                    'type': 'text',
                    'text': text
                }
            ]
        }
    ]
    # inserir prompt de sistema (opcional)
    if sys_prompt is not None:
      message.insert(0, {
          'role': 'system',
          'content': [
              {
                  'type': 'text',
                  'text': sys_prompt
              }
          ]
      })

    print(f'[INFO] Message send to model: {message}')
    # Preparação da entrada para o modelo
    inputs = self.processor_data(message)

    # tamanho da entrada (para removermos dos tokens de saída)
    input_len = inputs["input_ids"].shape[-1]

    # geração
    with torch.inference_mode():
      generation = self.model.generate(**inputs, max_new_tokens=max_tokens, do_sample=False)
      generation = generation[0][input_len:]

    # decodificação
    answer = self.processor.decode(generation, skip_special_tokens=True)
    # visualização
    if show_result:
      self.show_results(text, answer)

    return answer

  def send_image_and_text(self, text, image_inputs, sys_prompt=None, max_tokens=256, show_result=True):
    """Geração de texto com MedGemma tendo como entrada texto e imagens."""
    if "27b" in self.model_version.lower():
      raise ValueError("[ERRO] Essa função é inválida para esse modelo. O modelo MedGemma-27B é somente texto e não aceita imagens.")

    # Garante que image_inputs é uma lista
    if not isinstance(image_inputs, list):
        image_inputs = [image_inputs]

    # Carrega todas as imagens
    images = []
    for img_input in image_inputs:
        img = self.load_image(img_input)
        if img is not None:
            images.append(img)
        else:
            print(f"[ERRO] Falha ao carregar: {img_input}")

    if not images:
        raise ValueError("[ERRO] Nenhuma imagem válida foi carregada. Abortando requisição.")

    message = [
        {
            'role': 'user',
            'content': [
                {
                    'type': 'text',
                    'text': text
                }
            ]
        }
    ]
    if sys_prompt is not None:
      message.insert(0, {
          'role': 'system',
          'content': [
              {
                  'type': 'text',
                  'text': sys_prompt
              }
          ]
      })

    # Adiciona imagens ao prompt
    pos = 0 if sys_prompt is None else 1
    for img in images:
      message[pos]["content"].append({"type": "image", "image": img})

    inputs = self.processor_data(message, dtype=torch.bfloat16)

    # Geração
    outputs = self.model.generate(**inputs, max_new_tokens=max_tokens)
    response = self.processor.decode(outputs[0], skip_special_tokens=True)

    if show_result:
        print("\n[RESPOSTA DO MODELO]:\n", response)

    return response

  def load_image(self, image_input):
    """
      Carrega uma ou mais imagens (a partir de caminho, bytes ou BytesIO)
      e converte todas para RGB (formato necessário para o MedGemma).
      Retorna uma única imagem (PIL.Image) ou uma lista de imagens, conforme o input.
    """
    def _load_single_image(single_input):
        """Carrega uma única imagem e converte para RGB."""
        if isinstance(single_input, (str, os.PathLike)):
            if not os.path.exists(single_input):
                raise FileNotFoundError(f"O arquivo de imagem '{single_input}' não foi encontrado.")
            return Image.open(single_input).convert('RGB')

        elif isinstance(single_input, (bytes, bytearray, io.BytesIO)):
            if isinstance(single_input, (bytes, bytearray)):
                single_input = io.BytesIO(single_input)
            return Image.open(single_input).convert('RGB')

        else:
            raise TypeError("Cada elemento deve ser um caminho (str) ou um objeto de arquivo em memória.")

    try:
        # Caso o input seja uma lista ou tupla de imagens
        if isinstance(image_input, (list, tuple)):
            images = []
            for idx, img in enumerate(image_input):
                try:
                    loaded = _load_single_image(img)
                    images.append(loaded)
                except Exception as e:
                    print(f"[ERRO] Falha ao carregar imagem {idx + 1}: {e}")
            return images  # lista de imagens

        # Caso seja apenas uma imagem
        else:
            return _load_single_image(image_input)

    except Exception as e:
        print(f"[ERRO] Falha ao carregar imagem(s): {e}")
        return None

  def show_results(self, text, answer, image=None) -> None:
    display(Markdown(f"---\n\n**[ Requisição ]**\n\n{text}"))
    if image:
      if isinstance(image, list):
        for idx, img in enumerate(image, start=1):
          display(Markdown(f"**Imagem {idx}:**"))
          display(img)
      else:
          display(image)
    display(Markdown(f"---\n\n**[ MedGemma ]**\n\n{answer}\n\n---"))

  def select_device(self, device: str = None) -> str:
    """
      Verifica se o device está disponível e retorna a string correspondente.
      Caso device seja None, tenta usar GPU (CUDA) se disponível, senão CPU.
      Exibe avisos recomendando o uso de GPU.
    """
    # Caso o usuário tenha especificado manualmente
    if device is not None:
      if device.startswith("cuda") and not torch.cuda.is_available():
        print("[AVISO] CUDA foi especificado, mas nenhuma GPU está disponível. Alternando para CPU.")
        return "cpu"
      elif device == "cpu":
        print("[AVISO] O modelo será executado na CPU. Isso pode ser significativamente mais lento.")
        print("[SUGESTÃO] Considere usar uma GPU (CUDA) para acelerar a inferência.")
        return "cpu"
      else:
        return device

    # Seleção automática
    if torch.cuda.is_available():
      print("[INFO] GPU CUDA detectada. Utilizando GPU para melhor desempenho.")
      return "cuda"
    else:
      print("[AVISO] Nenhuma GPU detectada. O modelo será executado na CPU.")
      print("[SUGESTÃO] Considere usar uma GPU (CUDA) para acelerar a execução.")
      return "cpu"

  def config_model_params(self, device:str, use_quantization:bool) -> dict:
    # Definir os parâmetros do modelo baseado no ambiente do Colab

    # Caso dos modelos de 27B
    if '27b' in self.model_version:
      if 'A100' in torch.cuda.get_device_name(0):
        # Parâmetros utilizados pelo exemplo da própria Google
        model_kwargs = dict(
            torch_dtype=torch.bfloat16,
            device_map='auto',
            quantization_config=BitsAndBytesConfig(
                load_in_4bit=True # diminuição de alguns pesos para 4bits
            )
        )
      else:
        raise ValueError(
            "Runtime has insufficient memory to run a 27B variant."
            "Please select an A100 GPU."
        )
    # Caso dos modelos de 4B
    elif '4b' in self.model_version:
      if device == 'cpu': # apenas para depurar
        # Nesse modo a RAM fica muito próxima do seu limite
        model_kwargs = dict(
            device_map='cpu',
            # torch_dtype='float32',  # precisão máxima (está estourando o limite da RAM, 12.7GB)
            torch_dtype=torch.float16, # Deve utilizar essa precisão para não estourar a RAM
            low_cpu_mem_usage=True  # otimiza carregamento
        )
      elif device == 'cuda' and not use_quantization:
        # uso estimado de 10~12GB de VRAM
        # Ao utilizar esse modo a VRAM atingiu ~11GB de 15GB
        model_kwargs = dict(
            device_map='auto',
            torch_dtype=torch.bfloat16  # mais leve que float32
        )
      elif device == 'cuda' and use_quantization:
        # uso estimado de 4~6GB de VRAM
        # Utiliza ~4GB da VRAM
        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type='nf4'
        )
        model_kwargs = dict(
            device_map='auto',
            quantization_config=quant_config
        )
      else:
        print(f'No valid option')
        return None

    print(f'Parâmetros do Modelo:\n{model_kwargs=}')
    return model_kwargs

---
## HuggingFace login

In [ ]:
login()

---
## Experimentos

### MedGemma4B

In [ ]:
%%time
medgemma4b = UsingMedGemma()

Verificando funcionamento

In [ ]:
image_path = []
for root, _, files in os.walk('/content/'):
  for file in files:
    if file.lower().endswith('.jpg'):
      image_path.append(os.path.join(root, file))
print(image_path)

Português

In [ ]:
sys_prompt = 'Você é um especialista em radiografia'

In [ ]:
basic_prompt = 'Descreva o raio-x abaixo. Por favor, responda em português (PT-BR)'

In [ ]:
%%time
answer = medgemma4b.send_image_and_text(basic_prompt, image_path, sys_prompt)

English

In [ ]:
sys_prompt_en = 'You are an specialist in radiograph'

In [ ]:
basic_prompt_en = 'Describe this X-ray'

In [ ]:
%%time
answer_en = answer = medgemma4b.send_image_and_text(basic_prompt_en, image_path, sys_prompt_en)

---
Tradução de documentos

In [ ]:
text_prompt = """
Traduza para o português o laudo abaixo:

                                 FINAL REPORT
 EXAMINATION:  CHEST (PA AND LAT)

 INDICATION:  ___F with new onset ascites  // eval for infection

 TECHNIQUE:  Chest PA and lateral

 COMPARISON:  None.

 FINDINGS:

 There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral
 nodular opacities that most likely represent nipple shadows. The
 cardiomediastinal silhouette is normal.  Clips project over the left lung,
 potentially within the breast. The imaged upper abdomen is unremarkable.
 Chronic deformity of the posterior left sixth and seventh ribs are noted.

 IMPRESSION:

 No acute cardiopulmonary process.
"""

In [ ]:
%%time
answer_text = medgemma4b.send_only_text(text_prompt)

### MedGemma27B (text-only)

In [ ]:
%%time
medgemma27b = MedGemma('27b-text')

Copiando model e processor

In [ ]:
model27b = medgemma27b.return_model()

In [ ]:
processor27b = medgemma27b.return_processor()

In [ ]:
text_prompt = """
Traduza para o português o laudo abaixo:

                                 FINAL REPORT
 EXAMINATION:  CHEST (PA AND LAT)

 INDICATION:  ___F with new onset ascites  // eval for infection

 TECHNIQUE:  Chest PA and lateral

 COMPARISON:  None.

 FINDINGS:

 There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral
 nodular opacities that most likely represent nipple shadows. The
 cardiomediastinal silhouette is normal.  Clips project over the left lung,
 potentially within the breast. The imaged upper abdomen is unremarkable.
 Chronic deformity of the posterior left sixth and seventh ribs are noted.

 IMPRESSION:

 No acute cardiopulmonary process.
"""

In [ ]:
%%time
answer_text = medgemma27b.send_only_text(text_prompt)

In [ ]:
sys_prompt = 'Você é um tradutor de inglês para português brasileiro. Apenas traduza os textos que receber sem explicar.'

In [ ]:
%%time
answer_text = medgemma27b.send_only_text(text_prompt, sys_prompt)

In [ ]:
answer_text

### Prompt Engineering

Para estes experimentos irei utilizar o MedGemma4B, pois é menor e mais rápido.

In [ ]:
%%time
if 'medgemma4b' not in locals():
  medgemma = UsingMedGemma()
else:
  medgemma = medgemma4b

Prompt 1

In [ ]:
# Prompt 1
# Deixar claro que é um modelo tradudor de EN -> PT-BR e pedir para traduzir
sys_prompt_1 = 'Você é um modelo especialista em traduções de inglês para português brasileiro.'
text_prompt_1 = 'Traduza o texto abaixo para português brasileiro:\n'
# Texto a ser traduzido
text_en = '''
                                 FINAL REPORT
 EXAMINATION:  CHEST (PA AND LAT)

 INDICATION:  ___F with new onset ascites  // eval for infection

 TECHNIQUE:  Chest PA and lateral

 COMPARISON:  None.

 FINDINGS:

 There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral
 nodular opacities that most likely represent nipple shadows. The
 cardiomediastinal silhouette is normal.  Clips project over the left lung,
 potentially within the breast. The imaged upper abdomen is unremarkable.
 Chronic deformity of the posterior left sixth and seventh ribs are noted.

 IMPRESSION:

 No acute cardiopulmonary process.
'''
text_prompt_1 = text_prompt_1 + text_en

In [ ]:
ans_prompt1 = medgemma.send_only_text(text_prompt_1, sys_prompt_1, 500)

Prompt 2

In [ ]:
# Prompt 2
# Deixar claro que é um tradutor de EN -> PT-BR e também um especialista em laudos médicos.
sys_prompt_2 = 'Você é um tradutor de inglês para português brasileiro. Você também é um especialista em laudos médicos.'
text_prompt_2 = 'Traduza o texto abaixo para português brasileiro:\n'
# Texto a ser traduzido
# text_en = ''
text_prompt_2 = text_prompt_2 + text_en

In [ ]:
ans_prompt2 = medgemma.send_only_text(text_prompt_2, sys_prompt_2, 500)

### Traduções

Arquivo selecionado do Google Drive e JSON salvo na mesma pasta

Acessar conteúdos do GDrive

In [ ]:
drive.mount('/content/drive')

Caminho para arquivo json com os textos

In [ ]:
# _files_path = input('Insert json path:\n')
_files_path = '/content/drive/MyDrive/2025_2S_IA368HH/Projeto Final/SubDataset - MIMIC-CXR/30_samples_only_text/medical_texts.json'

Importar dados (JSON dos dados a serem traduzidos)

In [ ]:
def import_json_from_drive(file_path: str=_files_path) -> dict:
    """
    Importa um arquivo JSON do Google Drive (ou local) e retorna como dicionário.

    Args:
        file_path (str): Caminho completo do arquivo JSON.
                         Exemplo: '/content/drive/MyDrive/pasta/arquivo.json'

    Returns:
        dict: Dados carregados do arquivo JSON.
    """
    # Garante que o Google Drive está montado
    drive_path = '/content/drive'
    if not os.path.exists(drive_path):
        drive.mount(drive_path)
        print("Google Drive montado com sucesso.")

    # Verifica se o arquivo existe
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Arquivo não encontrado: {file_path}")

    # Lê o JSON
    with open(file_path, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)
            print(f"Arquivo JSON carregado com sucesso de: {file_path}")
            print(f"Total de registros: {len(data)}")
            return data
        except json.JSONDecodeError as e:
            raise ValueError(f"Erro ao decodificar o JSON: {e}")

In [ ]:
medical_texts = import_json_from_drive(_files_path)

In [ ]:
medical_texts['s50269882']

Tradução dos dados (com armazenamentos frequentes)

In [ ]:
translate_file_name = input('Enter the name of the json file that will contain the translated samples (Leave it as None if you want to use the default name):\n')
if len(translate_file_name) < 2:
  translate_file_name = f'medical_texts_27b.json' if 'medgemma27b' in locals() else f'medical_texts_4b.json'
if '.json' not in translate_file_name:
  translate_file_name += '.json'
print(translate_file_name)

In [ ]:
_save_path = os.path.join(os.path.dirname(_files_path), translate_file_name)
print(_save_path)

In [ ]:
def translate_function(model, dataset: dict, sys_prompt: str, user_prompt: str,
                       save_path: str = _save_path,
                       overwrite: bool = True,
                       max_tokens: int = 500,
                       save_every: int = 5):
    """
    Traduz textos usando o modelo e salva periodicamente o progresso no Google Drive.

    Args:
        model: Instância da classe do MedGemma (ou outro modelo).
        dataset: Dicionário com os textos em inglês.
        sys_prompt: Prompt de sistema (por ex: "Você é um tradutor médico...").
        user_prompt: Prompt usado para gerar a tradução.
        save_path: Caminho completo para salvar o JSON no Google Drive.
        overwrite: Se True, sobrescreve os elementos existentes.
        max_tokens: Número máximo de tokens de saída.
        save_every: Quantas traduções fazer antes de salvar o progresso.
    """

    # Garante que o diretório de saída exista
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    total = len(dataset)
    for i, id in enumerate(tqdm(dataset, total=total, ncols=100), start=1):
        # Pula se já estiver traduzido (se possuir elemento na chave portuguese)
        if 'portuguese' in dataset[id] and dataset[id]['portuguese'] and not overwrite:
            continue

        text_prompt = dataset[id]['english']

        try:
            # Tradução
            translated_text = model.send_only_text(
                text=f"{user_prompt}\n{text_prompt}",
                sys_prompt=sys_prompt,
                max_tokens=max_tokens,
                show_result=False
            )
            dataset[id]['portuguese'] = translated_text.strip()

        except Exception as e:
            print(f"[ERRO] Falha ao traduzir {id}: {e}")
            dataset[id]['portuguese'] = None # deixo vazia para tentar de novo

        # Salvamento periódico
        if i % save_every == 0 or i == total:
            try:
                with open(save_path, 'w', encoding='utf-8') as f:
                    json.dump(dataset, f, ensure_ascii=False, indent=2)
                print(f"[SALVO] Progresso salvo em: {save_path} ({i}/{total})")
            except Exception as e:
                print(f"[ERRO] Falha ao salvar JSON: {e}")
            # Espera um pouco para não sobrecarregar o Drive
            time.sleep(1)

    print(f"\n Tradução concluída. Arquivo final salvo em: {save_path}")
    return dataset

In [ ]:
dataset_with_translate = translate_function(medgemma4b,
                                            medical_texts,
                                            'Você é um tradutor de inglês para Português Brasileiro',
                                            'Traduza o texto abaixo:'
                                            )

### Tradução de algumas amostras

Varrendo as 30 amostras aleatórias com MedGemma 4B e 27B.

Obtendo JSON

In [ ]:
json_files = [
    file
    for root, _, files in os.walk('/content/')
    for file in files
    if file.lower().endswith('.json')
][0] # obter apenas o primeiro arquivo lido
print(json_files)

In [ ]:
with open(json_files, 'r', encoding='utf-8') as file:
  medical_data = json.load(file)

Tradução (aqui o modelo de medgemma utilizado depende de qual foi inicializado anteriormente)

In [ ]:
if 'medgemma27b' in locals():
  medgemma = medgemma27b
elif 'medgemma4b' in locals():
  medgemma = medgemma4b
else:
  medgemma = UsingMedGemma() # caso nenhuma esteja inicializada, será iniciada a versão 4B

In [ ]:
%%time
sys_prompt = 'Você é um tradutor de inglês para português brasileiro. Traduza os textos que receber.'
for id in tqdm(medical_data, ncols=100):
  text_prompt = medical_data[id]['english']
  medical_data[id]['portuguese'] = medgemma.send_only_text(text_prompt, sys_prompt, 500, False)

In [ ]:
medical_data

Salvando os textos no JSON original

In [ ]:
with open('medical_texts.json', 'w', encoding='utf-8') as f:
    json.dump(medical_data, f, ensure_ascii=False, indent=2)

---
## Consumo de GPU

In [ ]:
!nvidia-smi

Liberar VRAM e reduzir uso de GPU

In [ ]:
# gc.collect()
# torch.cuda.empty_cache()